## 필요 프레임워크 설치
1. Scrapy 설치


In [ ]:
# scrapy 설치
!pip install scrapy

## Project 생성

In [ ]:
# scrapy Project 생성 후 spiders 폴더에 sample.py 파일 생성
!scrapy startproject sample

In [ ]:
# settings.py 설정

# 실행명령어 설정
BOT_NAME = "news"

# 파일 저장방식 설정
FEED_FORMAT = "csv"

# 저장할 파일명 설정
FEED_URI = "조선일보_test.csv"

# 404 오류무시 설정
HTTPERROR_ALLOWED_CODES = [404]

In [ ]:
# items.py
# Spider에서 추출한 데이터를 구조화된 데이터로 변환해줌
import scrapy

class NewsItem(scrapy.Item):
    # 가져올 데이터 설정
    # 기사제목
    title = scrapy.Field()
    # 기자이름
    writer = scrapy.Field()
    # 작성날짜
    published_date = scrapy.Field()
    # 본문
    content = scrapy.Field()
    # 이미지
    imgs = scrapy.Field()
    pass

In [ ]:
# sample.py의 class 실행명령어 설정
import scrapy
import re # 정규표현식 사용

class testSpider(scrapy.Spider):
    # 실행명령어
    # settings.py에서 설정한 BOT_NAME과 같아야함(주의)
    name = 'test'

# 크롤링 과정

1. 크롤링할 메인 url 가져오기

In [ ]:
def start_request(self):
    # 헤더 설정
    headers= {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
        }
    
    # 크롤링할 메인 url 가져오기(조선일보)
    url = f'http://kid.chosun.com/list_kj.html?catid=1&pn={i}'

    # 결과를 리턴하고 종료하지 않고 crawl_page 실행
    yield scrapy.Request(url=url, callback=self.crawl_page, headers=headers)


2. 가져온 메인 url에서 크롤링할 url 저장


In [ ]:
def crawl_page(self,response):
    headers= {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
    }

    # 한 페이지에 10개의 기사가 존재
    for i in range(1, 11):
        # 예외처리(마지막 페이지에는 10개의 기사가 존재하지 않을 수도 있음.)
        try:
            # 크롤링할 url
            # xpath로 크롤링할 url 추출
            url = 'http://kid.chosun.com' + response.xpath(f'//*[@id="container"]/section[2]/article/ul/li[{i}]/div[2]/div[1]/a/@href')[0].extract()
            # url이 존재하면
            if url:
                # 종료하지 않고 parse 함수 호출
                yield scrapy.Request(url=url, callback=self.parse, headers=headers)
        except Exception as e:
            pass

3. 요청한 데이터를 파싱 후 필요한 데이터만 저장

In [ ]:
def parse(self, response):
    # 아이템 객체 생성
    item = NewsItem()

    # xpath로 저장할 데이터 추출
    # 기사 제목
    item['title'] = response.xpath('//*[@id="container"]/section[1]/div[2]/text()')[0].extract()
    
    # span_tag의 갯수 확인 -> 기자 이름이 없고 날짜만 있는 기사가 있음
    len_span_tags = len(response.xpath('//*[@id="container"]/section[1]/div[3]/div[1]/span').extract())
    # span_tag가 2개이상이면 기자 이름, 날짜가 존재
    if len_span_tags >= 2:
        # 기자 이름 
        item['writer'] = response.xpath('//*[@id="container"]/section[1]/div[3]/div[1]/span[1]/text()')[0].extract()
        # 작성 날짜
        item['published_date'] = response.xpath('//*[@id="container"]/section[1]/div[3]/div[1]/span[@class="date"]/text()')[0].extract().replace('\r\n', '')[9:]
    # 2개이하면 작성 날짜만 존재
    else:
        # 기자 이름 null
        item['writer'] = ''
        # 작성 날짜 
        item['published_date'] = response.xpath('//*[@id="container"]/section[1]/div[3]/div[1]/span[1]/text()')[0].extract().replace('\r\n', '')[9:]

    # 본문을 html 형식으로 가져옴
    content_div = response.xpath('//*[@id="article"]').get()
    # <> 태그를 정규표현식으로 제거
    content_regex = re.sub(r'<[^>]*>', '', content_div)
    # 본문
    item['content'] = content_regex

    # 이미지 리스트
    img_list = []
    # 정규표현식으로 img태그의 src만 찾기
    image_regexs = re.findall(r'<img[^>]*src="([^"]+)"[^>]*>', content_div)
    # 반복문으로 src링크를 하나씩 img_list에 저장
    for image_regex in image_regexs:
        img_list.append(image_regex)
    # 이미지
    item['imgs'] = img_list

    yield item